In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/dataset.csv')
df.shape

In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
df.info

In [ ]:
df.head()

In [ ]:
import re

In [ ]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+ ')
    result = hangul.sub('', text)
    return result

df['ko_text'] = [text_cleaning(x) for x in df['news_title']]
df.iloc[:30]

In [ ]:
from konlpy.tag import Okt
from collections import Counter

korean_stopwords_path = './data/korean_stopwords.txt'

with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
news_stopwords = ['종합', '송파구', '국민은행', '신한은행', '아이스크림', '코앞']
for stopword in news_stopwords:
    stopwords.append(stopword)

def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    nouns = [noun for noun in nouns if len(noun) > 1]
    nouns = [noun for noun in nouns if noun not in stopwords]
    return nouns

df['noun'] = [get_nouns(n) for n in df['ko_text']]
df.shapeape

In [ ]:
df.head()

In [ ]:
transactions = df['noun'].tolist()
transactions = [transaction for transaction in transactions if transaction]
transactions

In [ ]:
from apyori import apriori

In [ ]:
results = list(apriori(transactions, min_support=0.06, min_confidence=0.05, min_lift=1.0, max_length=2))
results

In [ ]:
columns = ['source', 'target', 'support']
network_df = pd.DataFrame(columns=columns)

for result in results:
    if len(result.items) == 2:
        items = [x for x in result.items]
        row = [items[0], items[1], result.support]
        series = pd.Series(row, index=network_df.columns)
        network_df = network_df.append(series, ignore_index=True)

network_df

In [38]:
corpus = "".join(df['ko_text'].tolist())

In [ ]:
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(corpus)
count = Counter(nouns)

remove_char_counter = Counter({x:count[x] for x in count if len(x) > 1})
remove_char_counter